# U.S. Business Broker Web Scraping Project

This notebook was used to execute a web scraping projet for a client on upwork.com. The project consisted of scraping information for each business broker contained in the website https://www.ibba.org/find-a-business-broker/ and exporting to an excel spreadsheet for analysis. The client wanted the Name, address, city, state, zip, phone number, and website name (if applicable) for each broker.

In [ ]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import requests

In [9]:
#Compile list of URL's for all states.
url_list = ['https://www.ibba.org/alabama-state/', 'https://www.ibba.org/arizona-state/', 
           'https://www.ibba.org/arkansas-state/', 'https://www.ibba.org/california-state/',
           'https://www.ibba.org/colorado-state/', 'https://www.ibba.org/connecticut-state/',
           'https://www.ibba.org/delaware-state/', 'https://www.ibba.org/florida-state/', 
           'https://www.ibba.org/georgia-state/', 'https://www.ibba.org/hawaii-state/',
           'https://www.ibba.org/idaho-state/', 'https://www.ibba.org/illinois-state/',
           'https://www.ibba.org/indiana-state/', 'https://www.ibba.org/iowa-state/',
           'https://www.ibba.org/kansas-state/', 'https://www.ibba.org/kentucky-state/',
           'https://www.ibba.org/louisiana-state/', 'https://www.ibba.org/maine-state/',
           'https://www.ibba.org/maryland-state/', 'https://www.ibba.org/massachusetts-state/',
           'https://www.ibba.org/michigan-state/', 'https://www.ibba.org/minnesota-state/',
           'https://www.ibba.org/mississippi-state/', 'https://www.ibba.org/missouri-state/',
           'https://www.ibba.org/montana-state/', 'https://www.ibba.org/nebraska-state/',
           'https://www.ibba.org/nevada-state/', 'https://www.ibba.org/new-hampshire-state/',
           'https://www.ibba.org/new-jersey-state/', 'https://www.ibba.org/new-mexico-state/',
           'https://www.ibba.org/new-york-state/', 'https://www.ibba.org/north-carolina-state/',
           'https://www.ibba.org/north-dakota-state/', 'https://www.ibba.org/ohio-state/',
           'https://www.ibba.org/oklahoma-state/', 'https://www.ibba.org/oregon-state/',
           'https://www.ibba.org/pennsylvania-state/', 'https://www.ibba.org/rhode-island-state/',
           'https://www.ibba.org/south-carolina-state/', 'https://www.ibba.org/tennessee-state/',
           'https://www.ibba.org/texas-state/', 'https://www.ibba.org/utah-state/',
           'https://www.ibba.org/vermont-state/', 'https://www.ibba.org/virginia-state/',
           'https://www.ibba.org/washington-state/', 'https://www.ibba.org/west-virginia-state/',
           'https://www.ibba.org/wisconsin-state/', 'https://www.ibba.org/wyoming-state/']

In [10]:
#--------PLEASE READ THESE COMMENTS-----------#
#This next for loop contains all the code to scrape the necessary information needed to complete this project.
#The basic premise of the loop is for each url, obtain the necessary information for each broker and ultimately...
#add this information to a data dictionary, which will then be converted to a pandas dataframe.
#Examples of lists defined in the loop are located under this code cell.

#Counter to count each broker.
count = 0
#Create data dictionary.
data = {}

#Loop through all url's to get information from each state.
for url in url_list:
    
    #Get source text
    source = requests.get(url)
    text = source.text
    
    #Create regex pattern to get matches for workers.
    pattern = re.compile("ibba.org\/broker-profile\/[a-zA-Z\/-]+\">([a-zA-Z\s-]+)<\/a><\/b><p>([\d\sa-zA-Z\.\-\/<>,#]*<p>)<b>([\d\s\-()\.+]*)")
    
    #use pattern to find all within text.
    matches = pattern.findall(text)
    
    #Take items from matches and put into arrays
    acsz = [] #Address city state zip
    names = [] #Names
    phones = [] #Phone numbers
    for item in matches:
        #Split on tags separating address from city, state.
        new_address = item[1].split('</p><p>')
    
        #Cut off last <p> tag from address
        new_address[-1] = new_address[-1][:-3]
    
        #Append to acsz list
        acsz.append(new_address)
    
        #Append name to names list
        names.append(item[0])
    
        #Append phone to phones list.
        phones.append(item[2])
    
    #Get address out of acsz
    address = []
    for item in acsz:
        #Use if address has two lines
        if len(item) == 3:
            #Append concatenation of address line 1 and 2
            address.append(item[0]+' '+item[1])
        else:
            #Just append first part of acsz
            address.append(item[0])
    
    #List comprehensions to get city, state, and zip from acsz.
    city = [item[-1].split(',')[0] for item in acsz]
    #Had to incorportate this complex list comprehension because one state (new york) has two words. had to account for this.
    state = [item[-1].split(',')[-1].split()[0]+' '+item[-1].split(',')[-1].split()[1] 
         if len(item[-1].split(',')[-1].split()) == 3 else item[-1].split(',')[-1].split()[0]
         for item in acsz]
    #Get zip code from acsz
    zipc = [item[-1].split(',')[-1].split()[-1] for item in acsz]
    
    #Create pattern to get websites for users that have websites
    web_pattern = re.compile('www.ibba.org\/broker-profile\/[\w\/.\"#,-<>\'\s()]*">([a-zA-z\s\-\.]*)[\w\/\.\"#,-<>\'\s()]*="([\w:\/.-]*)"')
    web_matches = web_pattern.findall(text)
    
    #Extract names from web matches
    web_match_names = [match[0] for match in web_matches]
    
    
    for i in range(len(names)):
        #For names in name list, create a dictionary for that name containing all persons data.
        name_data = {}
        name_data['Name'] = names[i]
        name_data['Address'] = address[i]
        name_data['City'] = city[i]
        name_data['state'] = state[i]
        name_data['zip'] = zipc[i]
        name_data['phone'] = phones[i]
        
        #Check if name is in web_match_names. If so, add website.
        for j in range(len(web_match_names)):
            #If name from names list is in website naems list:
            if names[i] == web_match_names[j]:
                #Add website name to website portion of name_data dictionary.
                name_data['Website'] = web_matches[j][1]
                break
            else:
                #Add 'no website' to name portion of name_data dictionary.
                name_data['Website'] = 'No Website'
        
        #Add data from person (name data) to data dictionary. Count is just the key for the dictionary entry.
        #By the end of this loop, count will equal the total number of brokers scraped from the website.
        data[count] = name_data
        count += 1
    
    

In [96]:
#matches example
matches

[('Dealflow Brokerage',
  '1716 Capitol Avenue</p><p>Cheyenne, Wyoming 82001<p>',
  '+14156551073'),
 ('Coran Woodmass',
  '690 S Highway 89, Suite 200</p><p>Jackson, Wyoming 83002<p>',
  '8008750335')]

In [97]:
#name example
names

['Dealflow Brokerage', 'Coran Woodmass']

In [98]:
#city example
city

['Cheyenne', 'Jackson']

In [99]:
#state example
state

['Wyoming', 'Wyoming']

In [100]:
#zip code example
zipc

['82001', '83002']

In [101]:
#web matches example
web_matches

[('Coran Woodmass', 'http://thefbabroker.com/')]

In [102]:
#Example of entry in data dictionary
data[0]

{'Address': '2200 University Blvd.',
 'City': 'Tuscaloosa',
 'Name': 'Clay Tindal',
 'Website': 'No Website',
 'phone': '2052089876',
 'state': 'Alabama',
 'zip': '35401'}

## Create pandas dataframe from data dictionary

In [11]:
df = pd.DataFrame(data).T[['Name', 'Address', 'City', 'state', 'zip', 'phone', 'Website']]

In [12]:
df = df.sort_values(['state', 'City']).reset_index(drop=True)

In [89]:
#Example of what final product will look like.
df.head()

,Name,Address,City,state,zip,phone,Website
0,Rebecca Walker-Jones,"118 North Ross Street, # 6",Auburn,Alabama,36830,334 456 8528,http://www.oldincsellsbusinesses.com
1,Don Smith,4 Office Park Cir Ste 207,Birmingham,Alabama,35223-2538,(205) 879-7220,No Website
2,Fletcher Brown,15 Office Park Cir Suite 213,Birmingham,Alabama,35223-2504,(205) 870-4809,http://www.brownbb.com
3,Lanier Chew,28218 Us Highway 98,Daphne,Alabama,36526-7014,(251) 626-4949,No Website
4,William Bruce,"210 South Mobile Street, Suite 17",Fairhope,Alabama,36532,(251) 990-5934,https://williambruce.org/


## Verify state names loaded correctly and no empty entries.

In [90]:
#Make sure all states loaded correctly. Looks good.
df.groupby('state').count()

,Name,Address,City,zip,phone,Website
state,,,,,,
Alabama,8,8,8,8,8,8
Arizona,16,16,16,16,16,16
Arkansas,8,8,8,8,8,8
California,246,246,246,246,246,246
Colorado,42,42,42,42,42,42
Connecticut,8,8,8,8,8,8
Delaware,2,2,2,2,2,2
Florida,112,112,112,112,112,112
Georgia,31,31,31,31,31,31


In [91]:
#Check .info() and .describe() methods. Looks like no missing entries.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 962 entries, 0 to 961
Data columns (total 7 columns):
Name       962 non-null object
Address    962 non-null object
City       962 non-null object
state      962 non-null object
zip        962 non-null object
phone      962 non-null object
Website    962 non-null object
dtypes: object(7)
memory usage: 52.7+ KB


## All looks good. Now time to export to excel.

In [14]:
from pandas import ExcelWriter

In [15]:
writer = ExcelWriter('US Business Brokers.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()